## Tune Model Parameters

Like in previous years, I'm going to use `RandomizedSearchCV` to tune the params. I tried grid search, but realised that tuning the models individually missed some params and functionality from the wrapper class (like filtering by minimum season, and adding column names to the `DataFrameConverter`). Also, grid search takes forever for estimators with even a moderate number of params (I tuned an `ExtraTreesRegressor` for 2 days, 5 hours). I was able to run a 72-combination grid in about 3.5 hours, so I'm thinking I can run 150 combinations while I'm at work, and that should be good enough.

Unlike last year, I'm going to optimise toward accuracy instead of MAE. My choice of MAE last year was due to a misunderstanding of a blog post showing that MAE has a greater probability of identifying the best model, because it returns a range of values rather than booleans, making it less susceptible to chance (i.e. being wrong by a few points results in a small MAE but is just as bad for accuracy as being wrong by 100 points). However, this means that MAE is a better metric for picking competition winners, but when the principle metric is accuracy, you're better off optimising for that than MAE.

## Code Setup

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import get_scorer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from mlxtend.regressor import StackingRegressor
import statsmodels.api as sm
from scipy import stats

from augury.ml_estimators import stacking_estimator
from augury.ml_estimators.base_ml_estimator import BaseMLEstimator, BASE_ML_PIPELINE
from augury.sklearn.model_selection import year_cv_split
from augury.sklearn.metrics import match_accuracy_scorer
from augury.sklearn.models import EloRegressor, TimeSeriesRegressor
from augury.sklearn.preprocessing import (
    TeammatchToMatchConverter,
    DataFrameConverter,
)
from augury.ml_data import MLData
from augury.settings import CV_YEAR_RANGE, SEED

np.random.seed(SEED)

In [3]:
data = MLData(train_year_range=(max(CV_YEAR_RANGE),))
data.data

2020-03-16 20:16:52,006 - kedro.io.data_catalog - INFO - Loading data from `model_data` (JSONLocalDataSet)...


team        oppo_team round_type  \
Adelaide         1991 1           Adelaide         Hawthorn    Regular   
                      2           Adelaide          Carlton    Regular   
                      3           Adelaide           Sydney    Regular   
                      4           Adelaide         Essendon    Regular   
                      5           Adelaide       West Coast    Regular   
...                                    ...              ...        ...   
Western Bulldogs 2020 19  Western Bulldogs         Richmond    Regular   
                      20  Western Bulldogs          Geelong    Regular   
                      21  Western Bulldogs  North Melbourne    Regular   
                      22  Western Bulldogs    Port Adelaide    Regular   
                      23  Western Bulldogs       West Coast    Regular   

                                   venue prev_match_oppo_team  \
Adelaide         1991 1    Football Park                    0   
                      2    Football Park             Hawthorn   
                      3           S.C.G.              Carlton   
                      4       Windy Hill               Sydney   
                      5          Subiaco             Essendon   
...                                  ...                  ...   
Western Bulldogs 2020 19          M.C.G.             St Kilda   
                      20   Kardinia Park             Richmond   
                      21       Docklands              Geelong   
                      22  Eureka Stadium      North Melbourne   
                      23       Docklands        Port Adelaide   

                         oppo_prev_match_oppo_team                      date  \
Adelaide         1991 1                  Melbourne 1991-03-22 03:56:00+00:00   
                      2                    Fitzroy 1991-03-31 03:56:00+00:00   
                      3                   Hawthorn 1991-04-07 03:05:00+00:00   
                      4            North Melbourne 1991-04-13 03:30:00+00:00   
                      5            North Melbourne 1991-04-21 05:27:00+00:00   
...                                            ...                       ...   
Western Bulldogs 2020 19                Gold Coast 2020-07-26 03:30:00+00:00   
                      20                 Melbourne 2020-08-01 06:55:00+00:00   
                      21                   Carlton 2020-08-09 05:40:00+00:00   
                      22                  Essendon 2020-08-15 04:05:00+00:00   
                      23               Collingwood 2020-08-22 03:30:00+00:00   

                          team_goals  team_behinds  score  ...  \
Adelaide         1991 1         24.0          11.0  155.0  ...   
                      2         12.0           9.0   81.0  ...   
                      3         19.0          18.0  132.0  ...   
                      4          6.0          11.0   47.0  ...   
                      5          9.0          11.0   65.0  ...   
...                              ...           ...    ...  ...   
Western Bulldogs 2020 19         0.0           0.0    0.0  ...   
                      20         0.0           0.0    0.0  ...   
                      21         0.0           0.0    0.0  ...   
                      22         0.0           0.0    0.0  ...   
                      23         0.0           0.0    0.0  ...   

                          oppo_rolling_prev_match_goals_divided_by_rolling_prev_match_goals_plus_rolling_prev_match_behinds  \
Adelaide         1991 1                                                 0.0                                                   
                      2                                                 0.0                                                   
                      3                                                 0.0                                                   
                      4                                                 0.0                               

In [4]:
X_train, y_train = data.train_data

In [5]:
se = stacking_estimator.StackingEstimator()
se

StackingEstimator(min_year=1965, name='stacking_estimator',
                  pipeline=StackingRegressor(meta_regressor=Pipeline(memory=None,
                                                                     steps=[('standardscaler',
                                                                             StandardScaler(copy=True,
                                                                                            with_mean=True,
                                                                                            with_std=True)),
                                                                            ('extratreesregressor',
                                                                             ExtraTreesRegressor(bootstrap=False,
                                                                                                 ccp_alpha=0.0,
                                                                                                 criterion='mse',
   

## Tune Model Params


### Base ML Model

Since this is just a Scikit-learn estimator, it's a bit more straightforward: use the defaults as a baseline, with a few values around them. I'm leaving out some of the less-important params to keep the total number of combinations reasonable.


### Elo Model

I've mostly been using the default params suggested by the blog post from which I got this model. I tried to do some param tuning when I first introduced it to my model, but at the time it was a feature builder rather than a model in its own right, so tuning params via the Scikit-learn interface was awkward to say the least, and didn't have much impact on the overall model performance.

As with the time-series statistical models, I don't understand some of these params as well as I should, and there are a lot of params to tune, so I'm keeping the number of values for each param to three or four, centered around the current default.


### Time-series model
I don't understand statistical models very well, so I've only tried ARIMA and Prophet (per `notebooks/3.0-time-series-models.ipynb`). It will probably be worth it to experiment with more time-series models next year, especially since my initial efforts were stymied by data not sorted by date, which caused confusing results. Since I haven't done much investigation of the strengths and weaknesses of the different models, and ARIMA seems to be a commonly-used model, I'll stick with that and just tune some of its basic parameters.

`order` has been a difficult one to pin down, especially the first param, as high values tend to raise errors in the model, because it's unable optimise during training, so I want to try a variety of values for that, while leaving the other two the same, since they have less-subjective statistical tests determining which values to use. `exog_cols` is another subjective one. I've focused on including information that would be lacking from a strict time-series data set that only has dates and score margins: `at_home` and any columns that give a general indication of the quality of the opponent. I want to keep the number of `exog_cols` short, because adding too many seems to raise model training errors similar to those that come from using too large a value for the first `order` param.


### Meta Estimator

Since they're both `ExtraTreesRegressor`s, I'm basing the param gid for the meta estimator on the results from the base estimator, namely removing values that didn't appear in anywhere in the top-10-performing combinations. I also removed `max_features`, because the meta estimator only has 3.

In [6]:
base_ml_grid = {
    # I trained the default model, and the unpruned tree depths ranged from 37 to 55
    "pipeline__pipeline-1__extratreesregressor__max_depth": np.arange(10, 51),
    "pipeline__pipeline-1__extratreesregressor__max_features": stats.uniform(loc=0.6, scale=0.4),
    "pipeline__pipeline-1__extratreesregressor__min_samples_leaf": np.arange(1, 3),
    "pipeline__pipeline-1__extratreesregressor__min_samples_split": np.arange(2, 5),
    "pipeline__pipeline-1__extratreesregressor__n_estimators": np.arange(75, 176),
}

elo_grid = {
    'pipeline__pipeline-2__eloregressor__home_ground_advantage': np.arange(5, 16),
    'pipeline__pipeline-2__eloregressor__k': stats.uniform(loc=15, scale=30),
    'pipeline__pipeline-2__eloregressor__m': stats.uniform(loc=110, scale=60),
    'pipeline__pipeline-2__eloregressor__s': stats.uniform(loc=200, scale=150),
    'pipeline__pipeline-2__eloregressor__season_carryover': stats.uniform(loc=0.3, scale=0.5),
    'pipeline__pipeline-2__eloregressor__x': stats.uniform(loc=0.3, scale=0.4)
}

ts_grid = {
    'pipeline__pipeline-3__timeseriesregressor__exog_cols': [
        ["at_home", "oppo_cum_percent"],
        ["at_home"],
        ["oppo_cum_percent"],
        ["at_home", "oppo_cum_percent", "oppo_line_odds"],
        ["at_home", "oppo_line_odds"],
        [],
    ],
    'pipeline__pipeline-3__timeseriesregressor__order': [
        (2, 0, 1),
        (4, 0, 1),
        (6, 0, 1),
        (8, 0, 1),
        (10, 0, 1)
    ],
    'pipeline__pipeline-3__timeseriesregressor__fit_method': ['css-mle', 'mle', 'css'],
    'pipeline__pipeline-3__timeseriesregressor__fit_solver': [
        'lbfgs',
        'bfgs',
        'newton',
        'nm',
        'cg',
        'ncg',
        'powell'
    ],
}

meta_grid = {
    "pipeline__meta_regressor__extratreesregressor__max_depth": np.arange(10, 51),
    "pipeline__meta_regressor__extratreesregressor__min_samples_leaf": np.arange(1, 3),
    "pipeline__meta_regressor__extratreesregressor__min_samples_split": np.arange(2, 5),
    "pipeline__meta_regressor__extratreesregressor__n_estimators": np.arange(75, 176),
}

grid = {
    # An earlier experiment showed cutting off data before 1965 improved performance slightly,
    # but I want to test that against the newer model structure and other param combinations
    'min_year': [1897, 1965],
    # Current param for tipresias_2019 is about 0.03
    'pipeline__pipeline-1__pipeline__correlationselector__threshold': stats.uniform(loc=0.01, scale=0.1),
    **base_ml_grid,
    **elo_grid,
    **ts_grid,
    **meta_grid,
}

In [7]:
rscv = RandomizedSearchCV(
    se,
    grid,
    # 150 took 18 hours, which is way too long. This should take about 12.
    n_iter=100,
    scoring={
        "neg_mean_absolute_error": get_scorer("neg_mean_absolute_error"),
        "match_accuracy": match_accuracy_scorer,
    },
    n_jobs=-1,
    cv=year_cv_split(X_train, CV_YEAR_RANGE),
    refit=False,
    verbose=1,
)

rscv.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
/usr/local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 32.9min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 143.1min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 373.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 407.1min finished


RandomizedSearchCV(cv=[(array([ True,  True,  True, ..., False, False, False]),
                        array([False, False, False, ..., False, False, False])),
                       (array([ True,  True,  True, ..., False, False, False]),
                        array([False, False, False, ..., False, False, False])),
                       (array([ True,  True,  True, ..., False, False, False]),
                        array([False, False, False, ..., False, False, False])),
                       (array([ True,  True,  True, ..., False, Fal...
                                                                                                  'powell'],
                                        'pipeline__pipeline-3__timeseriesregressor__order': [(2,
                                                                                              0,
                                                                                              1),
                                             

In [8]:
cvdf = (
    pd
    .DataFrame(rscv.cv_results_)
    .sort_values(['rank_test_match_accuracy', 'rank_test_neg_mean_absolute_error'])
    .filter(regex='mean_test_match_accuracy|mean_test_neg_mean_absolute_error|param_')
    .sort_index(axis=1)
)

cvdf.head(20)

,mean_test_match_accuracy,mean_test_neg_mean_absolute_error,param_min_year,param_pipeline__meta_regressor__extratreesregressor__max_depth,param_pipeline__meta_regressor__extratreesregressor__min_samples_leaf,param_pipeline__meta_regressor__extratreesregressor__min_samples_split,param_pipeline__meta_regressor__extratreesregressor__n_estimators,param_pipeline__pipeline-1__extratreesregressor__max_depth,param_pipeline__pipeline-1__extratreesregressor__max_features,param_pipeline__pipeline-1__extratreesregressor__min_samples_leaf,...,param_pipeline__pipeline-2__eloregressor__home_ground_advantage,param_pipeline__pipeline-2__eloregressor__k,param_pipeline__pipeline-2__eloregressor__m,param_pipeline__pipeline-2__eloregressor__s,param_pipeline__pipeline-2__eloregressor__season_carryover,param_pipeline__pipeline-2__eloregressor__x,param_pipeline__pipeline-3__timeseriesregressor__exog_cols,param_pipeline__pipeline-3__timeseriesregressor__fit_method,param_pipeline__pipeline-3__timeseriesregressor__fit_solver,param_pipeline__pipeline-3__timeseriesregressor__order
84,0.738910,-28.122987,1965,41,1,3,172,45,0.949369,2,...,7,23.5156,131.549,257.577,0.532906,0.634399,"[at_home, oppo_cum_percent]",css,bfgs,"(8, 0, 1)"
34,0.737930,-28.138940,1965,43,2,3,77,21,0.926554,2,...,13,32.6987,138.803,263.08,0.692334,0.555745,[oppo_cum_percent],css,powell,"(8, 0, 1)"
28,0.735017,-28.065005,1965,48,1,3,164,28,0.644479,1,...,14,37.2292,156.032,323.419,0.672106,0.572416,"[at_home, oppo_cum_percent, oppo_line_odds]",mle,lbfgs,"(8, 0, 1)"
35,0.735008,-28.037436,1965,47,2,3,156,39,0.821926,1,...,10,28.5273,164.628,244.694,0.561801,0.579057,"[at_home, oppo_line_odds]",mle,lbfgs,"(4, 0, 1)"
68,0.735003,-28.192278,1965,17,2,2,137,19,0.618778,1,...,7,39.7862,158.046,294.467,0.408162,0.507304,"[at_home, oppo_cum_percent, oppo_line_odds]",mle,lbfgs,"(10, 0, 1)"
50,0.734037,-27.932011,1965,46,1,3,110,32,0.968337,1,...,9,26.3792,154.655,230.837,0.693895,0.541489,[oppo_cum_percent],css,nm,"(2, 0, 1)"
65,0.733085,-27.990433,1965,38,2,2,87,27,0.808067,1,...,14,44.7102,110.014,227.821,0.491982,0.673036,[oppo_cum_percent],css,ncg,"(8, 0, 1)"
20,0.733085,-28.012419,1965,38,1,4,150,35,0.742389,1,...,9,17.8247,128.685,346.927,0.387665,0.306864,[at_home],css-mle,powell,"(8, 0, 1)"
3,0.733085,-28.043801,1965,38,1,3,164,43,0.75806,2,...,12,32.1133,141.25,344.176,0.722267,0.598928,"[at_home, oppo_line_odds]",mle,lbfgs,"(8, 0, 1)"
57,0.733061,-28.326940,1897,31,2,2,148,39,0.794972,2,...,7,30.3737,154.501,304.827,0.501275,0.387209,[],css,ncg,"(8, 0, 1)"


In [9]:
len(cvdf[cvdf['mean_test_match_accuracy'].isna()])

55

## Conclusion

With roughly half of the combinations being invalid and producing NaN results, trying to tune `statsmodels` models via cross-validation is a bad idea. I'm thinking that Scikit-learn algorithms are a bit more robust to wonky param combinations, so param tuning is a matter of squeezing a little bit of performance out of them. Traditional statistical models, however, are much more sensitive to changes to params, which requires a little more delicacy in the tuning process (i.e. use traditional statistical methods/tests to select parameters rather than brute-force cross-validation).

I also suspect having top-level params for the `StackingEstimator` class is less than ideal. It was fine for simpler models, but I should keep the different models & pipelines as independent as possible. Some might perform better with data from before 1965 dropped, some might perform worse. Better to move the `min_year` param to a preprocessing transformer class and tune it on a pipeline-by-pipeline basis.

Related to the above, I initially tried to tune the params of individual models in the ensemble, but then had problems (particularly with `ARIMA`) when tuning the `StackingEstimator`, because some of the data processing in the wrapper class changed the data that the sub-models fitted/predicted on. Part of the solution is being more careful about recreating the correct context when looking at individual models (I was in a hurry and made many mistakes that required re-runs), but simplifying the functionality of the wrapper class will also make this easier. In addition to reducing the number of potential combinations, I want to tune the models separately, because some of the weaker models (`TimeSeriesEstimator` and `EloRegressor`) don't impact the final predictions as much, so we risk making their params irrelevant, because the quality of their params gets overwhelmed by the impact of more-important params for more-important estimators. For example, if the best combination of Elo params coincides with sub-par params for the ExtraTrees meta-estimator, I wouldn't use them, because the final score would be sub-par.

Despite these misgivings, the footy season is almost upon us, so I'm going to call it "good enough", and revisit this for next season.

In [12]:
cvdf.to_json('notebooks/4.0-tune-params-results.json', indent=2, orient='records')

### Best param combination

```json
{
  "mean_test_match_accuracy": 0.7389099948,
  "mean_test_neg_mean_absolute_error": -28.1229870744,
  "param_min_year": 1965,
  "param_pipeline__meta_regressor__extratreesregressor__max_depth": 41,
  "param_pipeline__meta_regressor__extratreesregressor__min_samples_leaf": 1,
  "param_pipeline__meta_regressor__extratreesregressor__min_samples_split": 3,
  "param_pipeline__meta_regressor__extratreesregressor__n_estimators": 172,
  "param_pipeline__pipeline-1__extratreesregressor__max_depth": 45,
  "param_pipeline__pipeline-1__extratreesregressor__max_features": 0.9493692952,
  "param_pipeline__pipeline-1__extratreesregressor__min_samples_leaf": 2,
  "param_pipeline__pipeline-1__extratreesregressor__min_samples_split": 3,
  "param_pipeline__pipeline-1__extratreesregressor__n_estimators": 113,
  "param_pipeline__pipeline-1__pipeline__correlationselector__threshold": 0.0376827797,
  "param_pipeline__pipeline-2__eloregressor__home_ground_advantage": 7,
  "param_pipeline__pipeline-2__eloregressor__k": 23.5156358583,
  "param_pipeline__pipeline-2__eloregressor__m": 131.54906178,
  "param_pipeline__pipeline-2__eloregressor__s": 257.5770727802,
  "param_pipeline__pipeline-2__eloregressor__season_carryover": 0.5329064035,
  "param_pipeline__pipeline-2__eloregressor__x": 0.6343992255,
  "param_pipeline__pipeline-3__timeseriesregressor__exog_cols": [
    "at_home",
    "oppo_cum_percent"
  ],
  "param_pipeline__pipeline-3__timeseriesregressor__fit_method": "css",
  "param_pipeline__pipeline-3__timeseriesregressor__fit_solver": "bfgs",
  "param_pipeline__pipeline-3__timeseriesregressor__order": [
    8,
    0,
    1
  ]
}
```